In [97]:
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms as T
import torch.nn as nn

In [98]:
import os

import PIL.Image
import pandas as pd
import scipy.io
import torch
from torch.utils.data import Dataset

In [99]:
random_seed = 1
torch.manual_seed(random_seed)

In [139]:
def int_to_onehot(y, num_labels = 102):
    ary = np.zeros(num_labels)
    ary[y-1] = 1
    return ary

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = scipy.io.loadmat(annotations_file)['labels'][0]
        self.img_list = os.listdir(img_dir)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_list[idx])
        image = PIL.Image.open(img_path).convert('RGB')
        label = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, int_to_onehot(label)

In [147]:
num_classes = 102

resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(512, num_classes)

googlenet = models.googlenet(pretrained=True)
googlenet.fc = nn.Linear(1024, num_classes)

normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

In [148]:
print(resnet18)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [149]:
transform = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor()])
dataset = CustomImageDataset("imagelabels.mat", "102flowers/jpg", transform=transform)

train_set, val_set = torch.utils.data.random_split(dataset, [5733, 2456])

batch_size_train = 64
batch_size_test = 64

train_loader = torch.utils.data.DataLoader(train_set,
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(val_set,
  batch_size=batch_size_train, shuffle=True)

In [152]:
n_epochs = 50
learning_rate = 0.1
log_interval = 10

network = resnet18
optimizer = optim.SGD(network.parameters(), lr=learning_rate)
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
        output = network(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item()
        pred = output.max(1, keepdim=True)[1]
        targetNumber = target.max(1, keepdim=True)[1]
        correct += pred.eq(targetNumber.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  test_acc_torch = 100. * correct / len(test_loader.dataset)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    test_acc_torch))
  return test_acc_torch

In [153]:
test_acc_torch = test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test_acc_torch = test()

this is lable:51
this is lable:36
this is lable:52
this is lable:5
this is lable:13
this is lable:30
this is lable:31
this is lable:72
this is lable:8
this is lable:69
this is lable:27
this is lable:7
this is lable:98
this is lable:31
this is lable:12
this is lable:3
this is lable:33
this is lable:43
this is lable:30
this is lable:91
this is lable:20
this is lable:25
this is lable:36
this is lable:99
this is lable:83
this is lable:91
this is lable:17
this is lable:93
this is lable:72
this is lable:32
this is lable:97
this is lable:43
this is lable:92
this is lable:20
this is lable:23
this is lable:58
this is lable:83
this is lable:47
this is lable:83
this is lable:43
this is lable:42
this is lable:45
this is lable:77
this is lable:58
this is lable:93
this is lable:88
this is lable:66
this is lable:90
this is lable:1
this is lable:93
this is lable:58
this is lable:15
this is lable:65
this is lable:58
this is lable:39
this is lable:88
this is lable:94
this is lable:8
this is lable:26
thi

KeyboardInterrupt: 